# Scaling Computations using Parallel Computing

## Przemysław Szufel and Julian Samaroo

<a class="anchor" id="toc"></a>
## Table of contents
  

1. [Multithreading](#multithreading)
2. [Green threading](#green)
3. [Multi-processing and distributed computing](#multiprocessing)
4. [(background material) Parallelize via Single Instruction Multiple Data (SIMD)](#simd)

Before running this Jupyter notebook, set Julia's configured number of threads.
This should be done *before* actually running the `notebook()` command.
The number of threads can be also set up in Julia options in Visual Studio code (if this is used to run this notebook).
```
# run this code from Julia REPL just before starting Jupyter Notebook
ENV["JULIA_NUM_THREADS"]=4
```

In [ ]:
println("Number of threads that your Julia is running: $(Threads.nthreads()) threads")

In [ ]:
using Distributed
using BenchmarkTools

<a class="anchor" id="multithreading"></a>
### Multithreading
---- [Return to table of contents](#toc) ---


Let's learn multithreading with a simple 2D sum example. Here's the serial (single CPU) code:

In [ ]:
function ssum(x)
    r, c = size(x)
    y = zeros(c)
    for j in 1:c
        for i in 1:r
            # Note how we index over dimension 1 in the inner loop; this is important for performance!
            @inbounds y[j] += x[i, j]
        end
    end
    y
end

And here's it modified for multithreading (multiple CPUs at once):

In [ ]:
function tsum(x)
    r, c = size(x)
    y = zeros(c)
    Threads.@threads for j in 1:c
        for i in 1:r
            @inbounds y[j] += x[i, j]
        end
    end
    y
end


How do they perform?

In [ ]:
x = rand(1000,10000);

In [ ]:
@time ssum(x)
@time ssum(x);

In [ ]:
@time tsum(x)
@time tsum(x);

For most people's computers, `tsum` should be a bit faster (but it won't be 4x faster, as you might expect!)

### Locking mechanism for threads

When can you not do the same thing at the same time? Here's a bad multithreaded counter:

In [ ]:
function f_bad()
    x = 0
    Threads.@threads for i in 1:10^6
        x += 1
    end
    return x
end

In [ ]:
f_bad()

And here's the (fine) single-threaded counter:

In [ ]:
function f_add()
    x = 0 
    for i in 1:10^6
        x += 1
    end
    x
end

In [ ]:
f_add()

Ehh, those values shouldn't be different! Why is that?

Answer: `x += 1` isn't just one operation, it's 3! (read current x, increment value locally, write new x). On multiple threads, these might get misordered, leading to wrong results:

x = 0

T1: Read x (0)

T2: Read x (0)

T1: Increment local (1)

T2: Increment local (1)

T1: Write x (1)

T2: Write x (1)

So two increments became just one, uh oh!

In [ ]:
function f_atomic()
    x = Threads.Atomic{Int}(0)
    Threads.@threads for i in 1:10^6
        Threads.atomic_add!(x, 1)
    end
    return x[]
end

In [ ]:
f_atomic()

Yay, it works again! But is it fast?

In [ ]:
@btime f_add()
@btime f_atomic()

Hmmm, I guess it's actually not an improvement... Can we improve it?

In [ ]:
function f_spin()
    l = Threads.SpinLock()
    x = Ref(0)
    Threads.@threads for i in 1:10^6
        Threads.lock(l) do
            x[] += 1
        end
    end
    return x[]
end

function f_reentrant()
    l = ReentrantLock()
    x = Ref(0)
    Threads.@threads for i in 1:10^6
        Threads.lock(l) do
            x[] += 1
        end
    end
    return x[]
end


In [ ]:
@assert f_add() == f_atomic() == f_spin() == f_reentrant()

@btime f_add()
@btime f_atomic()
@btime f_spin()
@btime f_reentrant()

Well that's terrible! We have some lessons to learn:
- Multithreading is easy to access in Julia, but;
- ...multithreading isn't always a free performance win
- The compiler makes single-CPU code go *really* fast (usually)
- Correct multithreading can be surprisingly annoying and verbose

<a class="anchor" id="green"></a>
### Green threading (multitasking)
---- [Return to table of contents](#toc) ---

How does Julia's multithreading work? Let's forget the "multiple-CPU" idea and just see a simpler concept, called "multitasking". When does sleeping for 4 seconds not take 4 seconds?

In [ ]:
@time sleep(2)

In [ ]:
@time t = @async sleep(2)

In [ ]:
t

This `@async` macro creates a `Task`, which runs the `sleep(2)` code in the "background". This lets us start some long-running code, and then do something else while it runs. Useful! How far does this "background" magic go?

In [ ]:
function dojob(i)
    val = round(rand(), digits=2)
    sleep(val)   # this could be external computations with I/O
    i, val
end

In [ ]:
result = Vector{Tuple{Int,Float64}}(undef, 8);
@time for i=1:8
    result[i] = dojob(i)
end
result

In [ ]:
result = Vector{Tuple{Int,Float64}}(undef, 8);
@time for i=1:8
   @async result[i] = dojob(i)
end
result

Oops, that isn't right! Don't forget, just because something runs in the background, doesn't mean it also immediately completes. At some point we have to wait for it to finish!

In [ ]:
result = Vector{Tuple{Int,Float64}}(undef, 8);
@time @sync for i=1:8
   @async result[i] = dojob(i)
end
result

Very cool! And even cooler, Julia's multithreading is built on this idea. Unlike `@async` (which runs code on the same CPU where it's called), `Threads.@spawn` runs code on any available CPU that Julia has access to:

In [ ]:
result = Vector{Tuple{Int,Float64}}(undef, 8);
@time @sync for i=1:8
   Threads.@spawn result[i] = dojob(i)
end
result

Of course, as we said before, multithreading doesn't always give free performance gains, but multitasking generally is a convenient way to do multiple things on the same CPU, or on different CPUs, with minimal effort from you!

<a class="anchor" id="multiprocessing"></a>
### Multi-processing and distributed computing
--- [Return to table of contents](#toc) ---

Now let's go beyond a single computer - what if we want to use multiple computers? Or what if we want to use one computer, but separate our tasks into different Julia processes? Then we can use Distributed.jl!

In [ ]:
using Distributed

This code adds 4 workers (and avoids adding more)

In [ ]:
addprocs(max(0, 5-nprocs()));

In [ ]:
workers()

We've added some "workers" (independent Julia processes) on our local machine, which we can now control from IJulia/VSCode in many ways. Let's use the `@distributed` macro, which works similarly to the `Threads.@threads` macro (but for workers):

In [ ]:
function s_rand()
    n = 10^4
    x = 0.0
    for i in 1:n
        x += sum(rand(10^4))
    end
    x / n
end
 
@time s_rand()
@time s_rand()

In [ ]:
function p_rand()
    n = 10^4
    x = @distributed (+) for i in 1:n
        # the last line will be aggregated
        sum(rand(10^4))
    end
    x / n
end

@time p_rand()
@time p_rand()

It's parallel, and it's actually fast! (Of course, this is a really simple problem to parallelize). Like multithreading, it's also possible to just spawn a single "task" with workers:

In [ ]:
f = @spawnat 3 4+3

In [ ]:
fetch(f)

It's called a `Future`, but it's basically like a `Task`. By default, code runs on worker 1, but we can run code on any worker:

In [ ]:
function myf() 
    println("I am on worker ", myid()) # myid() returns our current worker
    rand()
end
myf()

In [ ]:
a = nothing
try 
    fetch(@spawnat 4 myf())
catch e
    println(e)
end

Oops! Code is also only evaluated on worker 1 by default, but we can use `@everywhere` to run code on all workers:

In [ ]:
@everywhere function myf() 
    println("I am on worker ", myid())
    rand()
end
fetch(@spawnat 4 myf())

#### A typical pattern for setting an intial state across workers

In [ ]:
using Distributed
@everywhere using Pkg
@everywhere Pkg.activate(".")
@everywhere using Distributed, Random, DataFrames

@everywhere function calc(x, y)
    2x + y
end

@everywhere function init_worker()    
   Random.seed!(myid())
    # reading initial data from files or other actions
end

@sync for wid in workers()
    @async fetch(@spawnat wid init_worker())
end


Typically results are collected to a `DataFrame`

In [ ]:
data = @distributed (append!) for (i, j) = vec(collect(Iterators.product(1:4, 1:3)))
    a = rand(1:499)
    b = rand(1:9)*1000
    c = calc(a, b)
    DataFrame(;i,j,a,b,c,procid = myid())
end

<a class="anchor" id="simd"></a>
### (background material) Parallelize via Single Instruction Multiple Data (SIMD)
---- [Return to table of contents](#toc) ---

A single CPU doesn't just have to do one thing at a time; with the power of SIMD, we can make our code faster by using parallelism built-in to CPUs:

In [ ]:
function dot1(x, y)
    s = 0.0
    for i in 1:length(x)
        @inbounds s += x[i]*y[i]
    end
    s
end

In [ ]:
function dot2(x, y)
    s = 0.0
    @simd for i in 1:length(x)
        @inbounds s += x[i]*y[i]
    end
    s
end

In [ ]:
x = 100*rand(10000)
y = 100*rand(10000)

@assert dot1(x, y) ≈ dot2(x, y)

@btime dot1($x, $y)
@btime dot2($x, $y)

Just beware that results aren't always exactly the same, due to floating point behavior:

In [ ]:
@show dot1(x, y) dot2(x, y)
@show dot1(x, y) == dot2(x, y)
@show dot1(x, y) ≈ dot2(x, y)